In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
train = pd.read_csv("/content/drive/MyDrive/기계학습론/train.csv")
train_Drop = train.drop(['belongs_to_collection', 
                         'homepage', 
                         'imdb_id', 
                         'original_language', 
                         'original_title', 
                         'overview', 
                         'poster_path', 
                         'production_companies',
                         'production_countries', 
                         'release_date', 
                         'spoken_languages', 
                         'status', 'tagline', 
                         'title', 'Keywords', 
                         'cast', 
                         'runtime',
                         'crew'],axis=1)

null_value_train = []
null_value_train.append(train_Drop['genres'].isnull())
train_drop = train_Drop.dropna()

In [3]:
test = pd.read_csv("/content/drive/MyDrive/기계학습론/test.csv")
test_Drop = test.drop(['belongs_to_collection', 
                         'homepage', 
                         'imdb_id', 
                         'original_language', 
                         'original_title', 
                         'overview', 
                         'poster_path', 
                         'production_companies',
                         'production_countries', 
                         'release_date', 
                         'spoken_languages', 
                         'status', 'tagline', 
                         'title', 'Keywords', 
                         'cast', 
                         'runtime',
                         'crew'],axis=1)

null_value_test = []
null_value_test.append(test_Drop['genres'].isnull())
test_drop = test_Drop.dropna()

In [4]:
null_index_train = []
for i in range(len(null_value_train[0])):
  if null_value_train[0][i] == True:
    null_index_train.append(int(i))

null_index_test = []
for i in range(len(null_value_test[0])):
  if null_value_test[0][i] == True:
    null_index_test.append(int(i))

In [5]:
train_genre_list = train_drop['genres'].tolist()
test_genre_list = train_drop['genres'].tolist()

In [6]:
train_id_list = []
for i in train_genre_list:

  string = i
  characters = "{ } [ ] '"

  string = ''.join(x for x in string if x not in characters)
  string = string.replace(',',':').split(':')

  for j in range(int(len(string)/4)):
    train_id_list.append(string[4*j+1])


test_id_list = []
for i in test_genre_list:

  string = i
  characters = "{ } [ ] '"

  string = ''.join(x for x in string if x not in characters)
  string = string.replace(',',':').split(':')

  for j in range(int(len(string)/4)):
    test_id_list.append(string[4*j+1])


In [7]:
train_unique_id = []
for i in train_id_list:
  if int(i) not in train_unique_id:
    train_unique_id.append(int(i))
train_unique_id.sort()
print(train_unique_id)

test_unique_id = []
for i in test_id_list:
  if int(i) not in test_unique_id:
    test_unique_id.append(int(i))
test_unique_id.sort()
print(test_unique_id)

[12, 14, 16, 18, 27, 28, 35, 36, 37, 53, 80, 99, 878, 9648, 10402, 10749, 10751, 10752, 10769, 10770]
[12, 14, 16, 18, 27, 28, 35, 36, 37, 53, 80, 99, 878, 9648, 10402, 10749, 10751, 10752, 10769, 10770]


In [8]:
for i in train_unique_id:
  train_drop["Genre " + str(i)] = int(0)

for i in test_unique_id:
  test_drop["Genre " + str(i)] = int(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [9]:
for index in range(len(train_drop)+len(null_index_train)):
  if index not in null_index_train:
    string = train_drop.loc[index]['genres']
    characters = "{ } [ ] '"

    string = ''.join(x for x in string if x not in characters)
    string = string.replace(',',':').split(':')

    for i in range(int(len(string)/4)):
      train_drop.loc[index, "Genre " + str(string[4*i+1])] = 1

for index in range(len(test_drop)+len(null_index_test)):
  if index not in null_index_test:
    string = test_drop.loc[index]['genres']
    characters = "{ } [ ] '"

    string = ''.join(x for x in string if x not in characters)
    string = string.replace(',',':').split(':')

    for i in range(int(len(string)/4)):
      test_drop.loc[index, "Genre " + str(string[4*i+1])] = 1


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [10]:
train_final = train_drop.drop(['genres'], axis = 1)
train_label = train_final.pop('revenue')

test_final = test_drop.drop(['genres'], axis = 1)

train_final['id'] = train_final['id'].astype(float)
train_final['budget'] = train_final['budget'].astype(float)
#train_final['revenue'] = train_final['revenue'].astype(float)
train_final['Genre 12'] = train_final['Genre 12'].astype(float)
train_final['Genre 14'] = train_final['Genre 14'].astype(float)
train_final['Genre 16'] = train_final['Genre 16'].astype(float)
train_final['Genre 18'] = train_final['Genre 18'].astype(float)
train_final['Genre 27'] = train_final['Genre 27'].astype(float)
train_final['Genre 28'] = train_final['Genre 28'].astype(float)
train_final['Genre 35'] = train_final['Genre 35'].astype(float)
train_final['Genre 36'] = train_final['Genre 36'].astype(float)
train_final['Genre 37'] = train_final['Genre 37'].astype(float)
train_final['Genre 53'] = train_final['Genre 53'].astype(float)
train_final['Genre 80'] = train_final['Genre 80'].astype(float)
train_final['Genre 99'] = train_final['Genre 99'].astype(float)
train_final['Genre 878'] = train_final['Genre 878'].astype(float)
train_final['Genre 9648'] = train_final['Genre 9648'].astype(float)
train_final['Genre 10402'] = train_final['Genre 10402'].astype(float)
train_final['Genre 10749'] = train_final['Genre 10749'].astype(float)
train_final['Genre 10751'] = train_final['Genre 10751'].astype(float)
train_final['Genre 10752'] = train_final['Genre 10752'].astype(float)
train_final['Genre 10769'] = train_final['Genre 10769'].astype(float)
train_final['Genre 10770'] = train_final['Genre 10770'].astype(float)

test_final['id'] = test_final['id'].astype(float)
test_final['budget'] = test_final['budget'].astype(float)
test_final['Genre 12'] = test_final['Genre 12'].astype(float)
test_final['Genre 14'] = test_final['Genre 14'].astype(float)
test_final['Genre 16'] = test_final['Genre 16'].astype(float)
test_final['Genre 18'] = test_final['Genre 18'].astype(float)
test_final['Genre 27'] = test_final['Genre 27'].astype(float)
test_final['Genre 28'] = test_final['Genre 28'].astype(float)
test_final['Genre 35'] = test_final['Genre 35'].astype(float)
test_final['Genre 36'] = test_final['Genre 36'].astype(float)
test_final['Genre 37'] = test_final['Genre 37'].astype(float)
test_final['Genre 53'] = test_final['Genre 53'].astype(float)
test_final['Genre 80'] = test_final['Genre 80'].astype(float)
test_final['Genre 99'] = test_final['Genre 99'].astype(float)
test_final['Genre 878'] = test_final['Genre 878'].astype(float)
test_final['Genre 9648'] = test_final['Genre 9648'].astype(float)
test_final['Genre 10402'] = test_final['Genre 10402'].astype(float)
test_final['Genre 10749'] = test_final['Genre 10749'].astype(float)
test_final['Genre 10751'] = test_final['Genre 10751'].astype(float)
test_final['Genre 10752'] = test_final['Genre 10752'].astype(float)
test_final['Genre 10769'] = test_final['Genre 10769'].astype(float)
test_final['Genre 10770'] = test_final['Genre 10770'].astype(float)




In [11]:
train_status = train_final.describe()
train_status = train_status.transpose()

label_status = train_label.describe()
label_status = label_status.transpose()

test_status = test_final.describe()
test_status = test_status.transpose()

normal_train = (train_final - train_status['mean']) / train_status['std']
normal_label = (train_label - label_status['mean']) / label_status['std']
normal_test = (test_final - test_status['mean']) / test_status['std']


In [12]:
def modeling():
  model = keras.Sequential([
                            layers.Dense(256, activation='relu', input_shape=[len(train_final.keys())]),
                            layers.Dense(256, activation='relu'),
                            layers.Dense(256, activation='relu'),
                            layers.Dense(256, activation='relu'),
                            layers.Dense(1)
  ])

  model.compile(loss='mse', optimizer='adam', metrics=['mae', 'mse'])

  return model


In [13]:
model = modeling()

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               6144      
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dense_3 (Dense)             (None, 256)               65792     
                                                                 
 dense_4 (Dense)             (None, 1)                 257       
                                                                 
Total params: 203,777
Trainable params: 203,777
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(
    normal_train, normal_label, epochs=1000, validation_split = 0.2, verbose = 0
)

In [16]:
normal_test = normal_test.fillna(0)
test_result = model.predict(normal_test)
test_result

array([[-3.52324903e-01],
       [-6.11658216e-01],
       [ 5.26033163e-01],
       [-1.46797061e-01],
       [-5.02471626e-01],
       [-6.15561664e-01],
       [-4.35408473e-01],
       [-3.83591950e-01],
       [-3.76572907e-01],
       [-3.96774888e-01],
       [-4.58562434e-01],
       [-1.70741916e-01],
       [ 4.96892303e-01],
       [-2.58417279e-01],
       [ 1.27308345e+00],
       [-4.98153567e-01],
       [ 1.68500566e+00],
       [ 5.74841797e-01],
       [ 2.71341276e+00],
       [-7.10280478e-01],
       [-4.64483738e-01],
       [-3.78396213e-01],
       [-8.28078687e-02],
       [-5.20750284e-01],
       [-6.85148776e-01],
       [ 9.87328112e-01],
       [-4.15756285e-01],
       [-4.48582530e-01],
       [-5.92683733e-01],
       [-4.22126830e-01],
       [ 1.70617294e+00],
       [-3.71113122e-01],
       [ 8.01739120e+00],
       [-5.01578152e-01],
       [ 1.28699183e-01],
       [ 1.36234617e+00],
       [-5.42644262e-01],
       [ 6.28070384e-02],
       [-4.2

In [17]:
temp_list = []
for value in test_result:
  temp_list.append(abs((value * label_status['std'] + label_status['mean'])).tolist())

prediction_list = []
for value in range(len(temp_list)):
  prediction_list.append(int(temp_list[value][0]))

test_id_list = test_final["id"].tolist()

for i in range(len(test_id_list)):
  test_id_list[i] = int(test_id_list[i])



In [18]:
col_name = ['id', 'revenue']
result_df = pd.DataFrame(index = [], columns=col_name)

result_df['id'] = test_id_list
result_df['revenue'] = prediction_list

result_df
result_df.to_csv("result.csv", mode='w', index=False)

In [19]:
result_df

,id,revenue
0,2501,18904344
1,2502,16801840
2,2503,139840656
3,2504,47202348
4,2505,1768536
...,...,...
493,2996,35139300
494,2997,21853000
495,2998,66643352
496,2999,121747792
